In [ ]:
# Télécharge les dataset
%%bash
mkdir -p breast_cancer
wget -O breast_cancer/breast-cancer-wisconsin.data https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data
mkdir -p nba
wget -O nba/data.csv https://raw.githubusercontent.com/fivethirtyeight/data/master/nba-elo/nbaallelo.csv

In [ ]:
# Importe les bibliothèques utilisées pendant ce tutoriel
import pandas as pd

# 0 - Rappels Python

- Pas de typage des variables


In [ ]:
a = 5
print(a)
a = "toto"
print(a)

- Indentation

In [ ]:
for i in range(0, 5):
    print(i)

- List indexing et slicing

In [ ]:
a = ["a", "b", "c", "d", "e"]
print(a[0]) # Index commence à 0
print(a[-1]) # On peut indexer à partir de la fin de la liste
print(a[1:-2]) # Slicing. La borne supérieure est exclue

- (presque) Tout est objet : ils ont des attributs et des fonctions accessibles via `.`

In [ ]:
a = [10, 11, 12, 13, 14]
a.insert(1, 20)
print(a)
a.sort()
print(a)

- Chaîne de caractères

In [ ]:
a = "toto"
b = 'toto'
print(a==b) # L'utilisation de "" ou '' est équivalent
print("toto")
print("'toto'") # La combinaison de "" et '' permet d'échapper

# I - Introduction à Pandas

Pandas est une bibliothèque Python permettant de manipuler et analyser des jeux de données tabulaires. Les données au format tabulaires sont omniprésentes, en particulier dans le domaine du libre. Par exemple, le gouvernement libère un grand nombre de données, enregistrée au format CSV, sur le site [data.gouv.fr](https://www.data.gouv.fr/fr/datasets/).

De manière plus "pratique", un grand nombre d'entreprises stockent leurs données dans une base de données SQL, ce qui n'est rien d'autre qu'un ensemble de tableaux !

Remarque :
- La partie I de ce TP est inspirée de ce très bon [tutoriel](https://realpython.com/pandas-python-explore-dataset/#using-loc-and-iloc) sur Pandas (en anglais).
- On utilisera dans un premier temps un jeu de données correspondant à des matchs de basket de la NBA.

## I.0 - Comment aurait-on fait en Python natif ?

Nous allons voir dans la prochaine section comment utiliser Pandas pour effectuer des requêtes rapidement et simplement dans un jeu de données tabulaire. Cependant, il est intéressant d'étudier très rapidement comment la même tâche serait effectuée en Python Natif.

- Premièrement, il est nécessaire d'ouvrir le fichier contenant le jeu de données :

In [ ]:
# Ouverture du fichier
with open("nba/data.csv", "r") as opened_file:
    lines = opened_file.readlines()

# Lines est une liste dont chaque élément est une ligne du fichier ouvert
for i in range(5):
    print(lines[i])

La première ligne du jeu de données contient la description de chaque colonne.

Les lignes suivantes décrivent les matchs contenus dans le jeu de données

- Deuxièmement, il est nécessaire d'enregistrer le header et les données séparément. De plus, il faut enregistrer les données dans une structure manipulable.

In [ ]:
# Récupération du nom de chaque colonne
header = lines[0].strip('\n').split(',')
print(header)

# Récupération des descriptions des matchs
data = []
for i in range(1, len(lines)):
    line = lines[i].strip('\n').split(',')
    data.append(line)

# Affichage des 5 premiers éléments
for i in range(5):
    print(data[i])

On remarque que `data` est une liste de liste. Chaque élément de `data` (la description d'un match) est une liste dont chaque élément est un attribut du match (ID, nom d'équipe, résultat, etc.)

Une fois que le header et les données sont sauvegardées en mémoire, on peut effectuer une requête dans le jeu de données. Par exemple, si l'on souhaite trouver le nombre de matchs gagnés dans les années 50 par les Celtics, la démarche est la suivante :

In [ ]:
# Enregistre les indices des colonnes utiles
year_id_index = header.index("year_id")
fran_id_index = header.index("fran_id")
game_result_index = header.index("game_result")

results = []
for i in range(len(data)): # Pour chaque ligne de data
    # Enregistre les données utiles pour évaluer les critères de la requête
    year_id = int(data[i][year_id_index])
    fran_id = data[i][fran_id_index]
    game_result = data[i][game_result_index]

    # Applique les critères de la requête
    played_in_fifties = (year_id >= 1950 and year_id < 1960)
    played_by_celtics = (fran_id == "Celtics")
    won = (game_result == "W")
    if played_in_fifties and played_by_celtics and won:
        results.append(data[i])

# Affiche le résultat de la requête
print(len(results))

Comme on vient de le voir, il est tout à fait possible d'effectuer des requêtes dans un jeu de données tabulaire en Python natif. Cependant, ça demande beaucoup de lignes de code, même pour une requête très simple. Les choses ne seront que plus complexes si nos données sont réparties dans plusieurs tableaux où si l'on espère grouper nos données selon certaines caractéristiques, etc.

**Nous allons donc voir comment Pandas permet de simplifier l'accès à un jeu de données tabulaire.**

## I.1 - Ouverture d'un fichier et affichage

Pandas offre une fonction d'ouverture de fichier pour chaque type de fichiers supporté par la bibliothèque (CSV, JSON, SQL, etc.). Les jeu de données que nous utiliserons dans ce tutoriel sont au format CSV. Nous utiliserons donc la fonction [read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), qui prend en argument la position du fichier et retourne la **Dataframe** correspondante.

In [ ]:
nba_df = pd.read_csv("nba/data.csv")
print(nba_df)
print(type(nba_df))

Les Dataframe sont des objets définis par Pandas. Elles correspondent à des **tableaux à 2 dimensions** (ligne/colonne). Les colonnes du tableau sont indéxées par leur nom (définis par le header du fichier), les lignes sont indexées par leur position.

Par défaut, lors de l'affichage d'une dataframe, toutes les colonnes ne sont pas affichées pour éviter les débordement hors de l'écran. Il est possible de modifier ce comportement pour forcer l'affichage de toutes les colonnes :

In [ ]:
pd.set_option("display.max.columns", None) # Modifie le comportement par défaut
print(nba_df)
pd.set_option("display.max.columns", 6) # Rétablit le comportement par défaut

**Astuces** :
- L'attribut **columns** d'une dataframe permet d'obtenir le nom de toutes les colonnes
- Les dataframe offrent deux fonctions permettant de limiter l'affichage de la dataframe aux premières ou dernières lignes : **head** et **tail**.

In [ ]:
print(nba_df.columns)

In [ ]:
print(nba_df.head())
print(nba_df.tail())

## II.2 - Accès aux éléments d'une Dataframe

### II.2.a - Théorie

Il y a de nombreuses manières différentes pour accéder aux données contenues dans une Dataframe ([loc](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html), [iloc](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html), [ix](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.DataFrame.ix.html), etc.). Pour simplifier les choses, nous utiliserons uniquement la méthode `loc` dans ce tutoriel.

Une dataframe possède un attribut `loc` qui permet d'accéder aux données de la dataframe de la manière suivante: `nba_df.loc[label_ligne, label_colonnee]`.

**Remarque** : Puisque nous avons laissé Pandas déterminer les labels des lignes automatiquement, le label d'une ligne est son indice.

**Exemple** :
- Par défaut, le label de la première ligne dans la dataframe est 0
- Par défaut, le label de la 63ème ligne dans la dataframe est 64

Pour accéder à la valeur contenue dans la 53ème ligne et la colonne `game_id`:

In [ ]:
print(nba_df.loc[54, 'game_id'])

Comme dans une liste, il est possible d'utiliser des `slices` pour accèder à plusieurs lignes ou colonnes.

**Remarque**: Contrairement aux listes natives de Python, l'indice de début et de fin sont inclus dans les valeurs retournées par un slice dans une Dataframe.

In [ ]:
print(nba_df.loc[5:10, "gameorder":"game_id"])

### II.2.b - Exercices

- Afficher toutes les colonnes des 5 premières lignes

In [ ]:
# À compléter

- Afficher uniquement la colonne `forecast` des 5 dernières lignes

In [ ]:
# À compléter

- Afficher uniquement les colonnes `game_id` et `game_result` des lignes d'indices 12 à 15

In [ ]:
# À compléter

## II.3 - Effectuer des requêtes dans une Dataframe

### II.3.a - Théorie



Comparer une Dataframe avec une valeur (entière, chaîne de caractères, etc.) permet d'obtenir un filtre correspondant à la comparaison de chaque cellule avec cette même valeur.

Par exemple, si l'on compare notre dataframe avec la chaîne de caractères `"W"`, nous obtiendrons un filtre. Ce filtre est une nouvelle dataframe de la même forme que la précédente, mais les cellules contiennent True ou False en fonction de la comparaison avec "W"

In [ ]:
print(nba_df == 'W')

On peut aussi filter de manière plus précise sur une ou plusieurs colonnes d'une dataframe, grâce à l'attribut `loc`:

In [ ]:
print(nba_df.loc[:, "forecast"] >= 0.5)

Enfin, il est aussi possible de combiner des filtres entre eux :

In [ ]:
filtre_1 = nba_df.loc[:, "forecast"] >= 0.5
print(filtre_1)

In [ ]:
filtre_2 = nba_df.loc[:, "game_result"] == "W"
print(filtre_2)

In [ ]:
result = filtre_1 & filtre_2
print(result)

Les opérateurs logiques et de comparaison disponibles dans Pandas sont :

| Symbole |           Signification          |
|:-------:|:--------------------------------:|
|  >, >=  | Comparaison numérique supérieure |
|  <, <=  | Comparaison numérique inférieure |
|    ==   |    Comparaison numérique égale   |
|    &    |            ET logique            |
|    \|   |            OU logique            |
|    ~    |           NON logique            |

En passant un filtre en tant que label de lignes à `loc`, le résultat est une dataframe ne contenant que les lignes pour lesquelles le filtre est `True`.

In [ ]:
print(nba_df.loc[result, :])



---


Reprenons maintenant l'exemple introduit au tout début du TP. Nous souhaitons savoir combien de matchs ont été gagné par les Celtics dans les années 50 :

In [ ]:
won = (nba_df.loc[:, "game_result"] == "W")
played_in_fifties = (nba_df.loc[:, "year_id"] >= 1950) & (nba_df.loc[:, "year_id"] < 1960)
played_by_celtics = (nba_df.loc[:, "fran_id"] == "Celtics")
mask = won & played_in_fifties & played_by_celtics

print(len(nba_df.loc[mask, :]))

On retrouve bien le même nombre que précédemment !

### II.3.b - Exercices

- Trouver le nombre de fois où les Huskies ont gagnés contre les Knicks

In [ ]:
# À compléter

**Réponse**: 3


---


- Quelle est la date du dernier match joué par les Cavaliers ?

In [ ]:
# À compléter

**Réponse**: Le 28 mars 1947.



## II.4 - Une idée des choses plus complexes rendues possibles par Pandas

Pandas permet des opérations plus complexes (de manière similaires à SQL) comme ["grouping"](https://pandas.pydata.org/docs/user_guide/groupby.html#groupby) ou ["joining"](https://pandas.pydata.org/docs/user_guide/merging.html). Dans ce tutoriel, on se concentrera uniquement sur grouping.

### II.4.a - Théorie

`Grouping` permet d'agglomérer les données en groupe selon une caractéristique donnée. Par exemple, on pourrait grouper les données par équipe :

In [ ]:
nba_df.groupby("fran_id")

On peut ensuite appliquer des opérations d'aggrégation comme "sum", "mean", "std" ou "count":

In [ ]:
print(nba_df.groupby("fran_id").count())

Cette fonctionnalité permet de répondre à des requêtes plus complexes et plus précise. Par exemple, si l'on souhaite **trouver le nombre de match gagnés par les deux meilleures équipes dans l'année 1982**:

In [ ]:
won = (nba_df.loc[:, "game_result"] == "W")
played_in_1982 = (nba_df.loc[:, "year_id"] == 1982)
mask = won & played_in_1982

print(nba_df.loc[mask, :].groupby("fran_id").count().sort_values("game_result"))

Résultat : Les équipes `Sixers` et `Celtics` ont toutes les deux gagnée 70 match en 1982 !

### II.4.b - Exercices

Trouver l'année ou les Thunder ont gagnés le plus de matchs

In [ ]:
# À compléter

Résultat: Les Thunder ont gagnés le plus de match lors de l'année 1996.

## II.5 - Visualisation des données

Pandas offre des [fonctions pour visualiser nos données](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.plot.html?highlight=plot#pandas.core.groupby.DataFrameGroupBy.plot) (en s'appuyant sur la bibliothèque matplotlib):

In [ ]:
nba_df.loc[mask, :].groupby("year_id").count().sort_values("game_result").loc[:, "game_result"].plot(kind="bar")

# II - Application pratique

## II.1 - Description du dataset

Ce dataset contient la description de 699 tumeurs du sein, ainsi qu'un diagnostic de la tumeur (bénigne vs maligne). Voici ce que chaque colonne du jeu de données décrit:



|    Description de la colonne   |          Plage de valeur         |
|:------------------------------:|:--------------------------------:|
|   1. Identifiant de la tumeur  |                 /                |
|       2. Clump Thickness       |              1 - 10              |
|   3. Uniformity of Cell Size   |              1 - 10              |
|   4. Uniformity of Cell Shape  |              1 - 10              |
|      5. Marginal Adhesion      |              1 - 10              |
| 6. Single Epithelial Cell Size |              1 - 10              |
|         7. Bare Nuclei         |              1 - 10              |
|       8. Bland Chromatin       |              1 - 10              |
|       9. Normal Nucleoli       |              1 - 10              |
|           10. Mitoses          |              1 - 10              |
|       **11. Classification**       | **(2 pour bénigne, 4 pour maligne)** |

In [ ]:
# Ouvre le dataset au format CSV avec la bibliothèque pandas
column_names = ["ID", "Clump Thickness", "Uniformity of Cell Size", "Uniformity of Cell Shape", "Marginal Adhesion", "Single Epithelial Cell Size", "Bare Nuclei", "Bland Chromatin", "Normal Nucleoli", "Mitoses", "Diagnostic"]
bc_df = pd.read_csv("breast_cancer/breast-cancer-wisconsin.data", header=None, names=column_names)
bc_df

## II.2 - Analyse des données

Dans le contexte de l'apprentissage automatique, il est parfois nécessaire de vérifier que le ratio d'exemple positif/négatif n'est pas trop biaisé dans un sens ou dans l'autre. Ici, cela correspond à vérifier que le nombre cas d'exemple de tuneur bégnigne est comparable à celui des tumeurs malignines.

De plus, il est souvent nécessaire de normaliser les données pour s'assurer que l'apprentissage des modèles s'effectue correctement. Ici, les données sont numériques, la normalisation consisterait donc à transformer les données pour qu'elles aient (après transformation) une moyenne de 0 et un écart-type de 1. Pour cela, nous avons besoin de mesurer la moyenne et l'écart-type de chaque variable de description des tumeurs.

Enfin, l'analyse des données doit nous permettre de distinguer si des données erronées se sont glissées dans le jeu de données. N'étant pas médecin, on se contentera ici de vérifier que toutes les données sont "bien formées". Ici, on se contentera de vérifier que toutes les données sont bien des nombres.

***TL;DR***: 3 Tâches à effectuer:
- Compter le nombre de tumeurs bénignes et malignes.
- Obtenir la moyenne et l'écart-type pour chacune des colonnes décrivant les tumeurs
- Vérifier que toutes les données sont bien des nombres

### II.2.a - Comptage du nombre de tumeurs bénignes et malignes.

On veut compter le nombre de tumeurs bénignes et malignes pour s'assurer que la répartition entre ces deux classes n'est pas trop déséquilibrée.





In [ ]:
# À compléter
nb_total =  # Nombre de tumeurs totales
nb_malign =  # Nombre de tumeurs malignes
nb_benign =  # Nombre de tumeurs malignes

print(f"nb_total={nb_total}, nb_malign={nb_malign}, nb_benign={nb_benign}, nb_malign+nb_benign={nb_malign+nb_benign}")

**Solution**:
- Le jeu de données décrit 699 tumeurs
- Il contient 241 tumeurs malignes
- Il contient 458 tumeurs bénignes

### II.2.b - Obtention des statistiques utiles à la normalisation

Si l'on souhaitait normaliser nos données, une normalisation appropriée serait de transformer nos données pour qu'elles aient une moyenne de 0 et un écart-type de 1.

Pour effectuer une telle normalisation, il est nécessaire de calculer la moyenne et l'écart-type pour chacune des colonnes décrivant une tumeur.

**Remarque**: La colonne *Bare Nuclei* peut poser problème, vous pouvez ne pas la considérer pour cette question

In [ ]:
# À compléter

**Solution**:
- Clump Thickness, mean=4.42, std=2.82
- Uniformity of Cell Size, mean=3.13, std=3.05
- Uniformity of Cell Shape, mean=3.21, std=2.97
- Marginal Adhesion, mean=2.81, std=2.86
- Single Epithelial Cell Size, mean=3.22, std=2.21
- *Bare Nuclei, mean=N/A, std=N/A*
- Bland Chromatin, mean=3.44, std=2.44
- Normal Nucleoli, mean=2.87, std=3.05
- Mitoses, mean=1.59, std=1.72

### II.2.c - Vérification que les les données sont "propres"

Dans le cadre de l'apprentissage automatique, il est nécessaire que les données utilisées soit "propres". C'est à dire qu'elles soient cohérentes et bien formées. Ici, on se contentera de vérifier que toutes les données sont bien **uniquement des nombres**.

**Aide**:
- La fonction [notnull](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.notnull.html) retourne une Dataframe contenant True pour les cellules ne contenant pas NaN, False sinon
- La fonction [to_numeric](https://pandas.pydata.org/docs/reference/api/pandas.to_numeric.html) retourne une nouvelle colonne où les cellules contenant des chaînes de caractères sont converties en nombres si elles correspondent à un nombre. Avec l'option `errors="coerce"`, les chaînes de caractères ne correspondant pas à des nombres sont remplacés par NaN.

In [ ]:
# À compléter

**Solution**
- Toutes les colonnes sauf *Bare Nuclei* ne contiennent que des chiffres.
- Les lignes d'indices 23, 40, 139, 145, 158, 164, 235, 249, 275, 292, 294, 297, 315, 321, 411, 617 contiennent le charactère '?' dans la colonne *Bare Nuclei*

## II.3 - Traitement des données

Pour entraîner un algorithme d'apprentissage automatique à prédire si une tumeur est bénigne ou maligne, il est nécessaire de diviser le jeu de données en deux. Le premier permet d'entraîner le modèle, le second permet de mesurer la précision du modèle.

Il est aussi nécessaire de séparer en deux listes différentes. La première contient la description des tumeurs, la deuxième contient uniquement le diagnostic (bénigne ou maligne).

Il faut aussi transformer la description du diagnostic de manière compréhensible par l'algorithme d'apprentissage automatique. 0 si la tumeur est bénigne, 1 si la tumeur est maligne.

Enfin, il est nécessaire d'avoir des données "propres". Dans ce jeu de données, certaines lignes contiennent une valeur erronnée représentée par "?". Il est nécessaire de supprimier ces lignes


**TL;DR:**
Quatres tâches à effectuer:
- "Nettoyer" le jeu de données
- Convertir la colone *'Diagnostic'* en **Bénigne=0**, **Maligne=1**
- Diviser le jeu de données en un sous-ensemble d'apprentissage (75% des données), et un sous-ensemble d'évaluation (25% des données)
- Extraire pour chaque sous-ensemble d'un côté la description de la tumeur, de l'autre côté le diagnostic de la tumeur.

### II.3.a - Nettoyage du jeu de données

Certaines valeurs du jeu de données ont été remplacée par '?' car non disponible lors de l'évaluation de la tumeur par le docteur. Les modèles d'apprentissages automatiques savent uniquement manipuler des nombres. Il est donc nécessaire de supprimer les lignes erronées dû à ces '?'

NB: En pratique, on essaye parfois de 'sauver' la ligne plutôt que de la supprimer entièrement. Ici, on pourrait tenter de remplacer '?' par la valeur moyenne de la colonne. On se contentera pour cet exercice de supprimer les lignes erronées.

In [ ]:
# À compléter

### II.3.b - Conversion de la colonne Diagnostic

On souhaite modifier la colonne *'Diagnostic'*. Actuellement, la description correspond à **Bénigne=2**, **Maligne=4**. On souhaite transformer la description en **Bénigne=0**, **Maligne=1**

In [ ]:
# À compléter

### II.3.c - Division du jeu de données en sous-ensemble d'entraînement et d'évaluation

On souhaite diviser le jeu de données total en deux sous-ensemble.
Un premier sous-ensemble contenant 75% des données servira à l'apprentissage du modèle.
Un second sous-ensemble contenant 25% des données servira à l'évaluation du modèle.

In [ ]:
# À compléter

### II.3.d - Séparation du diagnostique et de la description des tumeurs

Afin d'entraîner un modèle d'apprentissage automatique, nous devons séparer les entrées du modèle et ses sorties. Ici, on veut séparer la description d'une tumeur de son diagnostique.

*Attention, l'ID d'une tumeur ne fait pas partie de sa description !*

In [ ]:
# À compléter

## II.4 - Entraînement et évaluation du modèle

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(train_input_df, train_output_df)
model.score(test_input_df, test_output_df)